In [1]:
import sys
!{sys.executable} -m pip install numpy --user
!{sys.executable} -m pip install scipy --user
!{sys.executable} -m pip install ta --user
!{sys.executable} -m pip install datetime --user
!{sys.executable} -m pip install yfinance --user

!{sys.executable} -m pip install MetaTrader5 --user

!pip install MetaTrader5 --user


In [3]:
import warnings
from datetime import datetime
import pandas as pd
import MetaTrader5 as mt5
warnings.filterwarnings("ignore")
mt5.initialize()


class MT5:

   def get_data(symbol, n, timeframe=mt5.TIMEFRAME_D1):
        """ Función para importar los datos del símbolo elegido"""

        # Inicializamos la conexión si no estaba activa
        mt5.initialize()

        # Current date extract
        utc_from = datetime.now()

        # Import the data into a tuple
        rates = mt5.copy_rates_from(symbol, timeframe, utc_from, n)

        # Tuple to dataframe
        rates_frame = pd.DataFrame(rates)

        # Convert time in seconds into the datetime format
        rates_frame['time'] = pd.to_datetime(rates_frame['time'], unit='s')

        # Convert the column "time" in the right format
        rates_frame['time'] = pd.to_datetime(rates_frame['time'], format='%Y-%m-%d')

        # Set column time as the index of the dataframe
        rates_frame = rates_frame.set_index('time')
        return rates_frame

   def orders(symbol, lot, buy=True, id_position=None):
       """ Enviamos las órdenes """

        # Inicializamos la conexión si no estaba activa
       if mt5.initialize() == False:
           mt5.initialize()

       # Get filling mode 
       filling_mode = mt5.symbol_info(symbol).filling_mode - 1

       # Take ask price
       ask_price = mt5.symbol_info_tick(symbol).ask

       # Take bid price
       bid_price = mt5.symbol_info_tick(symbol).bid

       # Take the point of the asset
       point = mt5.symbol_info(symbol).point

       deviation = 20  # mt5.getSlippage(symbol)
       # **************************** Open a trade *****************************
       if id_position == None:

           # Buy order Parameters
           if buy:
               type_trade = mt5.ORDER_TYPE_BUY
               sl = ask_price*(1-0.01)
               tp = ask_price*(1+0.01)
               price = ask_price

           # Sell order Parameters
           else:
               type_trade = mt5.ORDER_TYPE_SELL
               sl = bid_price*(1+0.01)
               tp = bid_price*(1-0.01)
               price = bid_price

           # Open the trade
           request = {
               "action": mt5.TRADE_ACTION_DEAL,
               "symbol": symbol,
               "volume": lot,
               "type": type_trade,
               "price": price,
               "deviation": deviation,
               "sl": sl,
               "tp": tp,
               "magic": 234000,
               "comment": "python script order",
               "type_time": mt5.ORDER_TIME_GTC,
               "type_filling": filling_mode,
           }
           # send a trading request
           result = mt5.order_send(request)
           result_comment = result.comment

       # **************************** Close a trade *****************************
       else:
           # Buy order Parameters
           if buy:
               type_trade = mt5.ORDER_TYPE_SELL
               price = bid_price

           # Sell order Parameters
           else:
               type_trade = mt5.ORDER_TYPE_BUY
               price = ask_price

           # Close the trade
           request = {
               "action": mt5.TRADE_ACTION_DEAL,
               "symbol": symbol,
               "volume": lot,
               "type": type_trade,
               "position": id_position,
               "price": price,
               "deviation": deviation,
               "magic": 234000,
               "comment": "python script order",
               "type_time": mt5.ORDER_TIME_GTC,
               "type_filling": filling_mode,
           }

           # send a trading request
           result = mt5.order_send(request)
           result_comment = result.comment
       return result.comment

   def resume():
      """ Return the current positions. Position=0 --> Buy """
      # Initialize the connection if there is not
      mt5.initialize()

      # Define the name of the columns that we will create
      columns = ["ticket", "position", "symbol", "volume"]

      # Go take the current open trades
      current = mt5.positions_get()

      # Create a empty dataframe
      summary = pd.DataFrame()

      # Loop to add each row in dataframe
      # (Can be ameliorate using of list of list)
      for element in current:
           element_pandas = pd.DataFrame([element.ticket,
                                          element.type,
                                          element.symbol,
                                          element.volume],
                                         index=columns).transpose()
           summary = pd.concat((summary, element_pandas), axis=0)

      return summary


   def run(symbol, long, short, lot):

        # Inicializamos la conexión si no estaba activa
        if mt5.initialize() == False:
            mt5.initialize()

        # Choose your  symbol
        print("------------------------------------------------------------------")
        print("Date: ", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        print("SYMBOL:", symbol)

        # Initialize the device
        current_open_positions = MT5.resume()
        # Buy or sell
        print(f"BUY: {long} \t  SHORT: {short}")

        """ Close trade eventually """
        # Extraction type trade
        try:
            position = current_open_positions.loc[current_open_positions["symbol"]==symbol].values[0][1]

            identifier = current_open_positions.loc[current_open_positions["symbol"]==symbol].values[0][0]
        except:
            position= None
            identifier = None

        print(f"POSITION: {position} \t ID: {identifier}")

        # Close trades
        if long==True and position==0:
            long=False

        elif long==False and position==0:
            res = MT5.orders(symbol, lot, buy=True, id_position=identifier)
            print(f"CLOSE LONG TRADE: {res}")

        elif short==True and position ==1:
            short=False

        elif short == False and position == 1:
            res = MT5.orders(symbol, lot, buy=False, id_position=identifier)
            print(f"CLOSE SHORT TRADE: {res}")

        else:
            pass


        """ Buy or short """
        if long==True:

            res = MT5.orders(symbol, lot, buy=True, id_position=None)
            print(f"OPEN LONG TRADE: {res}")

        if short==True:
            res = MT5.orders(symbol, lot, buy=False, id_position=None)
            print(f"OPEN SHORT TRADE: {res}")

        print("------------------------------------------------------------------")

   def close_all_night():
        result = MT5.resume()
        for i in range(len(result)):
            before =  mt5.account_info().balance
            row = result.iloc[0+i:1+i,:]
            if row["position"][0]==0:
                res = MT5.orders(row["symbol"][0], row["volume"][0], buy=True, id_position=row["ticket"][0])

            else:
                res = MT5.orders(row["symbol"][0], row["volume"][0], buy=False, id_position=row["ticket"][0])

In [5]:
MT5.get_data("EURUSD",100)

,open,high,low,close,tick_volume,spread,real_volume
time,,,,,,,
2024-04-24,1.07006,1.07141,1.06779,1.06980,29785,0,0
2024-04-25,1.06982,1.07402,1.06784,1.07290,34908,0,0
2024-04-26,1.07283,1.07527,1.06740,1.06922,33357,0,0
2024-04-29,1.06940,1.07337,1.06865,1.07200,38304,0,0
2024-04-30,1.07197,1.07353,1.06630,1.06655,36356,0,0
...,...,...,...,...,...,...,...
2024-09-04,1.10426,1.10951,1.10310,1.10815,37351,2,0
2024-09-05,1.10808,1.11195,1.10746,1.11092,39191,1,0
2024-09-06,1.11090,1.11543,1.10655,1.10850,46100,0,0


In [17]:
MT5.orders("EURUSD",0.01)

'Request executed'

In [23]:
#MT5.orders("EURUSD",0.01,id_position=2825558912)

'Request executed'

In [25]:
#MT5.resume()

""


In [9]:
mt5.account_info()


AccountInfo(login=86418026, trade_mode=0, leverage=1, limit_orders=200, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=False, balance=3001.82, credit=0.0, profit=0.0, equity=3001.82, margin=0.0, margin_free=3001.82, margin_level=0.0, margin_so_call=50.0, margin_so_so=30.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='Toni Vives', server='MetaQuotes-Demo', currency='EUR', company='MetaQuotes Ltd.')